In [8]:
from helpers_2 import *

from surprise import Dataset, Reader, SVD
from surprise.model_selection import train_test_split
from surprise import accuracy
from helpers_2 import load_dataset
import pandas as pd
import numpy as np

def ndcg_at_k(predictions, k=10):
    def dcg(relevance_scores, k):
        return sum(rel/np.log2(i+2) for i, rel in enumerate(relevance_scores[:k]))

    user_predictions = {}
    for uid, _, true_r, est, _ in predictions:
        if uid not in user_predictions:
            user_predictions[uid] = []
        user_predictions[uid].append((true_r, est))
    
    ndcg_values = []
    for uid in user_predictions:
        user_predictions[uid].sort(key=lambda x: x[1], reverse=True)
        relevance_scores = [pred[0] for pred in user_predictions[uid]]
        idcg = dcg(sorted(relevance_scores, reverse=True), k)
        dcg_value = dcg(relevance_scores, k)
        ndcg_values.append(dcg_value/idcg if idcg > 0 else 0)
    
    return np.mean(ndcg_values)

def mean_reciprocal_rank(predictions):
    user_rankings = {}
    for uid, _, true_r, est, _ in predictions:
        if uid not in user_rankings:
            user_rankings[uid] = []
        user_rankings[uid].append((true_r, est))
    
    mrr_values = []
    for uid in user_rankings:
        user_rankings[uid].sort(key=lambda x: x[1], reverse=True)
        for i, (true_r, est) in enumerate(user_rankings[uid]):
            if true_r > 0:
                mrr_values.append(1/(i+1))
                break
    
    return np.mean(mrr_values)

def precision_recall_at_k(predictions, k=10, threshold=4.0):
    user_predictions = {}
    for uid, _, true_r, est, _ in predictions:
        if uid not in user_predictions:
            user_predictions[uid] = []
        user_predictions[uid].append((true_r, est))
    
    precisions = []
    recalls = []
    for uid in user_predictions:
        user_predictions[uid].sort(key=lambda x: x[1], reverse=True)
        true_positive = np.sum([1 for true_r, est in user_predictions[uid][:k] if true_r >= threshold])
        relevant_items = np.sum([1 for true_r, est in user_predictions[uid] if true_r >= threshold])
        recommendations_count = min(k, len(user_predictions[uid]))

        precisions.append(true_positive/recommendations_count)
        recalls.append(true_positive/relevant_items if relevant_items != 0 else 0)
    
    return np.mean(precisions), np.mean(recalls)


In [10]:
# Load datasets
df_articles, df_clicks = load_dataset()
dataframe = df_clicks.merge(df_articles, left_on='click_article_id', right_on='article_id')
dataframe = dataframe[['user_id', 'article_id', 'category_id']]

series = dataframe.groupby(['user_id', 'category_id']).size()
user_rating_matrix = series.to_frame()
user_rating_matrix = user_rating_matrix.reset_index()
user_rating_matrix.rename(columns={0: 'rate'}, inplace=True)
user_rating_matrix["rate"].value_counts()

reader = Reader(rating_scale=(1, 10))
_x = user_rating_matrix.loc[user_rating_matrix.rate > 1]
data = Dataset.load_from_df(_x[['user_id', 'category_id', 'rate']], reader)

print('We have selected', len(_x), 'interactions.')
trainset, testset = train_test_split(data, test_size=0.25)
print('Test set length:', len(testset))
print('Train set length:', len(_x) - len(testset))

algo = SVD()
algo.fit(trainset)
predictions = algo.test(testset)

print('Number of predictions in Test set:', len(predictions))
accuracy.rmse(predictions)

# Calculate metrics
ndcg10 = ndcg_at_k(predictions, k=10)
mrr = mean_reciprocal_rank(predictions)
precision, recall = precision_recall_at_k(predictions, k=10, threshold=7)  # Assuming rating>=7 is relevant

print('NDCG@10:', ndcg10)
print('MRR:', mrr)
print('Precision@10:', precision)
print('Recall@10:', recall)

We have selected 503616 interactions.
Test set length: 125904
Train set length: 377712
Number of predictions in Test set: 125904
RMSE: 7.4957
NDCG@10: 0.9803608097842309
MRR: 1.0
Precision@10: 0.048361442078664205
Recall@10: 0.08253718291118296
